# Tracks Exploring

## Imports

In [2]:
import sqlite3
import pandas as pd

In [3]:
# Load the 'artists' table into a DataFrame
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file"""
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print("Connected to database")
        return True, conn
    except sqlite3.Error as e:
        print(f"Failed to connect: {e}")
        return False, None

# path to database file
status, conn  = create_connection("playlist_analysis.db")

if status:
    df_tracks = pd.read_sql_query("SELECT * FROM tracks;", conn)
else:
    print("Failed to establish a connection.")

# Display first few rows of the DataFrame to get an overview
df_tracks.head()

Connected to database


,t_uri,t_name,t_duration_ms,artist_uri,album_uri,popularity,explicit,preview_url,last_updated
0,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,Lose Control (feat. Ciara & Fat Man Scoop),226863,spotify:artist:2wIVse2owClT7go1WT98tk,spotify:album:6vV5UrXcfyQD1wu4Qo2I9K,67.0,1.0,https://p.scdn.co/mp3-preview/253a76c453026570...,None
1,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,Toxic,198800,spotify:artist:26dSoYclwsYLMAKD3tpOr4,spotify:album:0z7pVBGOD7HCIB7S8eLkLI,81.0,0.0,https://p.scdn.co/mp3-preview/6de2791f84c1d637...,None
2,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,Crazy In Love,235933,spotify:artist:6vWDO969PvNqNYHIOW5v0m,spotify:album:25hVFAxTlDvXbx2X2QkUkE,20.0,0.0,https://p.scdn.co/mp3-preview/bcd82e2e27606ec0...,None
3,spotify:track:1AWQoqb9bSvzTjaLralEkT,Rock Your Body,267266,spotify:artist:31TPClRtHm23RisEBtV3X7,spotify:album:6QPkyl04rXwTGlGlcYaRoW,76.0,0.0,https://p.scdn.co/mp3-preview/a4cdd210220f23d3...,None
4,spotify:track:1lzr43nnXAijIGYnCT8M8H,It Wasn't Me,227600,spotify:artist:5EvFsr3kj42KNv97ZEnqij,spotify:album:6NmFmPX56pcLBOFMhIiKvF,0.0,0.0,None,None


In [4]:
missing = df_tracks.isnull().sum()
print(missing)

t_uri                 0
t_name                0
t_duration_ms         0
artist_uri            0
album_uri             0
popularity           38
explicit             38
preview_url      153627
last_updated     297900
dtype: int64


In [4]:
import requests
import time
import json
import sqlite3
from datetime import datetime

# Create a connection to the database
def create_connection(db_file):
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except sqlite3.Error as e:
        print(e)
        return None
    
def alter_tracks_table(conn):
    try:
        cur = conn.cursor()
        cur.execute("ALTER TABLE tracks ADD COLUMN last_updated TEXT;")
        conn.commit()
        print("Tracks table altered successfully. Added column last_updated.")
    except sqlite3.Error as e:
        print(f"SQL error: {e}")

# Get all track URIs from the database
def get_all_track_uris(conn):
    cur = conn.cursor()
    cur.execute("SELECT t_uri FROM tracks;")
    return cur.fetchall()

# Create Spotify access token
def get_spotify_access_token(client_id, client_secret):
    auth_url = 'https://accounts.spotify.com/api/token'
    auth_response = requests.post(auth_url, {
        'grant_type': 'client_credentials',
        'client_id': client_id,
        'client_secret': client_secret,
    })
    auth_response_data = auth_response.json()
    return auth_response_data['access_token'], auth_response_data['expires_in']

# Check if the token is about to expire
def is_token_expired(token_time, expires_in):
    current_time = time.time()
    return current_time - token_time >= expires_in - 10  # 10 seconds buffer

# Update track info in the database
def update_track_info(conn, track_uris, access_token):
    headers = {'Authorization': f'Bearer {access_token}'}
    track_ids = [uri.split(":")[-1] for uri in track_uris]
    
    r = requests.get(f"https://api.spotify.com/v1/tracks?ids={','.join(track_ids)}", headers=headers)
    
    # Handle rate limiting
    if r.status_code == 429:
        retry_after = int(r.headers.get('Retry-After', 60))  # Default to 60 seconds if header is missing
        print(f"Rate limit exceeded. Retrying in {retry_after} seconds.")
        time.sleep(retry_after)
        update_track_info(conn, track_uris, access_token)  # Recursive retry
    
    # Handle other status codes
    
    elif r.status_code != 200:
        print(f"Failed to update tracks : Code: {r.status_code} : {r.text}")
        return
    
    # Update the database
    else:
        tracks_data = r.json()['tracks']
        cur = conn.cursor()
        
        for track_data in tracks_data:
            if track_data is None:
                print("Skipping None track data")
                continue
                
            track_uri = f"spotify:track:{track_data['id']}"
            popularity = track_data.get('popularity', None)
            explicit = track_data.get('explicit', None)
            preview_url = track_data.get('preview_url', None)

        cur.execute("UPDATE tracks SET popularity = ?, explicit = ?, preview_url = ?, last_updated = ? WHERE t_uri = ?",
            (popularity, explicit, preview_url, datetime.now().strftime('%Y-%m-%d %H:%M:%S'), track_uri))


        conn.commit()
        print(f"Status code for batch: {r.status_code}")


# Main function
if __name__ == '__main__':
    # Initialize
    conn = create_connection('playlist_analysis.db')
    client_id = 'your_client_id'
    client_secret = 'your_client_secret'
    access_token, expires_in = get_spotify_access_token(client_id, client_secret)
    token_time = time.time()  # Store the current time

    # Add new columns to the tracks table
    alter_tracks_table(conn)

    # Get all track URIs
    all_track_uris = get_all_track_uris(conn)
    total_tracks = len(all_track_uris)
    print(f"Total tracks to update: {total_tracks}")
    
    
    # Initialize rate limiting variables
    rate_limit_window = 25  # seconds
    max_requests = 20  # Maximum number of requests in rate_limit_window
    request_times = []

    # Update tracks in batches of 50
    for i in range(0, len(all_track_uris), 50):
        batch = all_track_uris[i:i + 50]

        # Check if the token is about to expire
        if is_token_expired(token_time, expires_in):
            print("Refreshing token...")
            access_token, expires_in = get_spotify_access_token(client_id, client_secret)
            token_time = time.time()

        # Check rate limit
        current_time = time.time()
        request_times = [t for t in request_times if current_time - t < rate_limit_window]
        if len(request_times) >= max_requests:
            sleep_time = rate_limit_window - (current_time - min(request_times))
            print(f"Rate limit reached. Sleeping for {sleep_time:.2f} seconds.")
            time.sleep(sleep_time)

        update_track_info(conn, [uri[0] for uri in batch], access_token)
        print(f"Successfully updated track info for batch {i} to {i+49}")

        time.sleep(0.2)  # to respect rate limits
    
    conn.close()

Tracks table altered successfully. Added column last_updated.
Total tracks to update: 301546
Status code for batch: 200
Successfully updated track info for batch 0 to 49
Status code for batch: 200
Successfully updated track info for batch 50 to 99
Status code for batch: 200
Successfully updated track info for batch 100 to 149
Status code for batch: 200
Successfully updated track info for batch 150 to 199
Status code for batch: 200
Successfully updated track info for batch 200 to 249
Status code for batch: 200
Successfully updated track info for batch 250 to 299
Status code for batch: 200
Successfully updated track info for batch 300 to 349
Status code for batch: 200
Successfully updated track info for batch 350 to 399
Status code for batch: 200
Successfully updated track info for batch 400 to 449
Status code for batch: 200
Successfully updated track info for batch 450 to 499
Status code for batch: 200
Successfully updated track info for batch 500 to 549
Status code for batch: 200
Succe

In [5]:
# path to database file
status, conn  = create_connection("playlist_analysis.db")

if status:
    df_tracks = pd.read_sql_query("SELECT * FROM tracks;", conn)
else:
    print("Failed to establish a connection.")

missing = df_tracks.isnull().sum()
print(missing)

Connected to database
t_uri                 0
t_name                0
t_duration_ms         0
artist_uri            0
album_uri             0
popularity           38
explicit             38
preview_url      153627
last_updated     297900
dtype: int64


In [6]:
# Convert 't_duration_ms' to seconds
df_tracks['t_duration_sec'] = df_tracks['t_duration_ms'] / 1000
df_tracks.describe()

,t_duration_ms,popularity,explicit,t_duration_sec
count,3.015460e+05,301508.000000,301508.000000,301546.000000
mean,2.407204e+05,16.803740,0.122222,240.720411
std,1.108625e+05,18.574412,0.327543,110.862513
min,0.000000e+00,0.000000,0.000000,0.000000
25%,1.915180e+05,0.000000,0.000000,191.518000
50%,2.256530e+05,11.000000,0.000000,225.653000
75%,2.694530e+05,31.000000,0.000000,269.453000
max,6.348017e+06,91.000000,1.000000,6348.017000
